# Lab 11 · Security and Hardening

*This lab notebook provides guided steps. All commands are intended for local execution.*

## Objectives
- Production CORS settings are tightened.
- Payload limits and allow-lists are described.
- SlowAPI rate limiting and prompt-injection checklists are provided.

## What will be learned
- Environment-specific CORS strategies are reinforced.
- Input validation patterns are documented for safety.
- Prompt-injection mitigations are rehearsed for RAG systems.

## Prerequisites & install
The following commands are intended for local execution.

```bash
cd ai-web/backend
. .venv/bin/activate
pip install slowapi
```

## Step-by-step tasks
### Step 1: Security settings module
Security utilities are collected for reuse.

In [ ]:
from pathlib import Path
security_path = Path("ai-web/backend/app/security.py")
security_path.write_text('''from slowapi import Limiter
from slowapi.util import get_remote_address
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware

SAFE_ORIGINS = ["https://example.com"]
ALLOWED_FILES = {".txt", ".md"}
limiter = Limiter(key_func=get_remote_address, default_limits=["60/minute"])


def configure_security(app: FastAPI):
    app.state.limiter = limiter
    app.add_middleware(
        CORSMiddleware,
        allow_origins=SAFE_ORIGINS,
        allow_credentials=True,
        allow_methods=["GET", "POST"],
        allow_headers=["Content-Type", "Authorization"],
    )
''')
print("Security module was written.")

### Step 2: Payload guard
A payload guard demonstrates enforcing limits and allow-lists.

In [ ]:
from pathlib import Path
limits_path = Path("ai-web/backend/app/guards.py")
limits_path.write_text('''from fastapi import HTTPException, UploadFile

MAX_PAYLOAD_BYTES = 1024 * 1024
ALLOWED_EXTENSIONS = {".txt", ".md", ".pdf"}


def enforce_size(data: bytes):
    if len(data) > MAX_PAYLOAD_BYTES:
        raise HTTPException(status_code=413, detail="Payload size limit was exceeded.")


def enforce_extension(upload: UploadFile):
    if not any(upload.filename.endswith(ext) for ext in ALLOWED_EXTENSIONS):
        raise HTTPException(status_code=400, detail="File extension was not allowed.")
''')
print("Payload guard was documented.")

### Step 3: Prompt-injection checklist
A checklist is referenced for the RAG pipeline.

A checklist is maintained:
- External instructions are screened for deny-list phrases.
- Citations are verified prior to response emission.
- Sensitive tools are disabled when citations are missing.

## Validation / acceptance checks
```bash
# locally
curl http://localhost:8000/health
```
- The hardened configuration preserves the health endpoint for monitoring.
- React development mode shows the described UI state without console errors.

## Homework / extensions
- Rate limiting thresholds are tuned for production traffic patterns.
- Security review notes are captured alongside deployment manifests.